# LAB Session 10 (21/12/2020)

First we generate the Higgs mass spectrum, with signal and background, and perform a fits to binned data. Then you can repeat the generation to emulate multiple experimens and perform some statistical analysis.

In order to help you with the excerise, the notebook provides the necessary steps to obtain a correct spectrum and fit it. Then you will be asked to answer a few questions by repeating some of the steps and make some plots.

The main steps are
- generate Gaussian signal
 - plot the distribution
- generate exponential background
 - plot the distribution 
- Plot the distribution of data by stacking signal and background
- Obtain binned data from histogram
- fit binned data with `curve_fit`
- compute the signal significance defined as $S/\sqrt B$

At this point you can repeat the above steps to perform a few studies and make some plots. The details of each exercise are provided at the end.

# Modules for this exercise

- `scipy.optimize` for fitting
- `scipy.integrate` for integration

In [ ]:
import numpy as np
import matplotlib.pyplot as pl
import scipy.stats as stats
from scipy import optimize as op  
from scipy import integrate

# Parameters
- Num. of signal events
- Num. of background events
- Higgs mass
- Higgs width due to detector resolution (intrinsic width is negligible)

In [ ]:
n_sig_0 = 100
n_bkg_0 = 10000

m_sig = 125 # GeV
w_sig = 0.02 # in %
w_sig *= m_sig

# Histogram data
boundaries and number of bins

In [ ]:
x_min = 0 # GeV
x_max = 200 # GeV
n_bins = 100
bin_width = (x_max-x_min)/n_bins # GeV
print('%d bins from %.1f to %.1f'%(n_bins, x_min, x_max))
print('bin width: %.2f GeV'%(bin_width))

## Generate signal sample

`n_sig` events with a Gaussian distribution around Higgs mass

## Plots signal distribution

Use [matplotlib.pyplot.hist](https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.hist.html)

In [ ]:
%matplotlib notebook


## Generate background sample

`n_bkg` events with an exponential distribution. For example, you can use [numpy.random.exponential](https://numpy.org/doc/stable/reference/random/generated/numpy.random.exponential.html)

Choose an exponential parameter that populates bins with tens of events for mass of 200 GeV. for example use `scale=80` in [numpy.random.exponential]

## Plots background distribution
Use [matplotlib.pyplot.hist](https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.hist.html)

Useful options:
- _bins=n_ : fix number of bins to _n_
- _hist=options_ : for example _hist='step'_ to show empty histogram
- _range=(min,max)_ : fix the boundaries of data in histogram. 
  - **NB**: this is different than using _xlim(min,max)_ which is only for visualization


## Stack signal and background data in the same histogram

If you have two data arrays `signal` and `background` try the following options to stack signal on top of the background

```python
pl.hist( (background,signal), bins=n_bins, label=('backgroud', 'signal'), color=('red','blue'), stacked=True , 
        histtype='step', range=(x_min,x_max) )
```

## Merge signal and background samples

The data collected in an experiment contains both signal and background. We have simulated the two samples separately and they can be merged with [numpy.concatenate](https://numpy.org/doc/stable/reference/generated/numpy.concatenate.html)

```python
a = np.array(...)
b = np.array(...)

data = np.concatenate( (a,b) )
```
Note ow the first positional argument of the function is a tuple of numpy arrays.

## Binned data

The data sample so far is unbinned: we have a number of signal and background events. 

Now we can create a binned version to be fitted. This is achieved with the `pyplot.hist` function which returns a list of counts for each bin and the bin boundaries
```python

binned_data, bins, _ = pl.hist( data, bins=n_bins, label='all data', color='black', histtype='step', range=(x_min,x_max) )

```

plot the merged data and make sure it is indeed the sum of the two sub-samples.

## `bins` list

The list `bins` contains `n_bins + 1` values: edge of the first bin ... edge of the last bin. You can verify this easily

In [ ]:
print(len(bins))
print(bins[:10])
print(bins[-10:])

## center of bins (data along x axis)

Using a comprehension, prepare a list `center_bins` from `bins` with length of `n_bins` which contains the center of each bin of the histogram

In [ ]:
print(len(center_bins))
print(center_bins[:5])
print(center_bins[-5:])

## bin count (data along y axis)

Verify that the list `binned data` should have length `n_bins` 

In [ ]:
print(len(binned_data))
print(binned_data[:10])
print(binned_data[-10:])

## Fitting the histogram

We want to fit the binned data (`binned_data` as a function of `center_bins`) using the [optimize.curve_fit](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html) function

The fir model has to be the sum of a Gaussian and an exponential.

Recall that [optimize.curve_fit](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html) has the following structure
```python
curve_fit( function, x_data, y_data , ... )
```

Define three functions with proper arguments
1. `sig(x, Norm, mean, width)` for the signal Gaussian
2. `bkg(x, Norm, alpha)` for the background exponential
3. `total(x,NormSig, mean, width, NormBkg, alpha)` sum of signal and background

There are a total of 5 parameters to be fitted from data

## Plot fit function

As a sanity check you can plot the 3 functions and make sure they are implemented correctly.
```python

pl.plot(x, sig(...), 'b-', label='signal')

pl.plot(x, bkg(...), 'r--', label='background', )

pl.plot(x, total(...), 'navy', label='total')

```

## Initial conditions for the fit

Since there are 5 parameters and the data varies a lot across the spectrum, it is always a good practice to provide some initial values for the parameters to be fitted. This is done with the argument `p0` in [optimize.curve_fit](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html)
```python
pars, pars_cov = op.curve_fit(..., p0=[500, m_sig, w_sig, 1000, alpha])
```

Recall that `pars` are the fitted parameters (in the order they appear in the definition of the fit function) and `pars_cov` is their covariance matrix that can be used to compute the uncertainty on each fitted parameter

## Fitted parameters
- print the parameters
- print the covariance matrix

are the fitted values close to the generated values?

In [ ]:
print('==== fitted parameters ===\n', pars)
print('==== covariance matrix\n', pars_cov)

## Plot the histogram and fit

- plot th data histogram
- Overlay the fitted function (total) with the fitted parameters
- overlay the background function with the fitted parameters
- print the fitted parameters with proper legend on the screen


# Signal Significance

The signal significance is defined as $S/\sqrt{B}$ where $S = \int_a^b sig(m)\, dm$ and $B = \int_a^b bkg(m)\, dm$ with $a = m_{\rm fitted} - 3\sigma_{\rm fitted}$ and $b = m_{\rm fitted} + 3\sigma_{\rm fitted}$

In order to compute the definite integral, you can use [scipy.integrate.quad](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.quad.html)

For example to integrate `func` in `[x_min, x_max]` 
```python 

def poly1(x, a, b):
    return a*x + b

x_min = 3
x_max =  10

m = 2
q = 3

res = scipy.integrate.quad( poly1, x_min, x_max, args=(m,q) )
```

`res` is a list with two elements
- the value of the integral `res[0]`
- its uncertainty `res[1]`

## compute S

In [ ]:
fitted_mass = pars[1]
fitted_width = pars[2]

a = fitted_mass-3.*fitted_width
b = fitted_mass+3.*fitted_width

## compute B

## compute significance and print the result

# Exercise 1

1. Fix the number of background events $N_B = 10000$ and vary the number of signal events $N_S$ in 
 `[10, 50, 100, 500, 1000]`
  - plot the signficance as a function of $N_S$
  - You can now change $N_B$ and remake the same plot and show the data for different values of $N_B$
  - how does the significance depend on $N_S$ and $N_B$ ?
 

# Exercise 2

2. Fix $N_B = 10000$ and $N_S = 200$. Vary the mass width between `[0.01, 0.2, 0.05, 0.10, 0.20, 0.50]`
  - plot the significance as a function of the mass width 
  - which mass width provide the best signal significance?

# Exercise 3


3. Fix $N_B = 10000$ and $N_S = 200$ and repeat the experiments 1000 times
  - for each experiment generate the number of signal and background events from a Poisson distribution
    - mean of signal events: 200
    - mean of background events: 10000
  - compute the significance for each experiment
  - plot the distribution of significance for all experiments
  - plot the distribution of $m_{fitted} - m_H$ for all experiments
  - fit to a Gaussian and print the fitted $\mu$ and $\sigma$